### Load Input Datasets

In [ ]:
# Calculate the date range for the last 7 days
from datetime import datetime, timedelta


date_range = 7
end_date = datetime.now() - timedelta(days=1)
# Add in correlationid to join search and click, tracking from 3/18/2025
start_date = max(datetime(2025, 3, 18), end_date - timedelta(days=date_range))
date_range = (end_date - start_date).days
print(f"Start date: {start_date}")
print(f"End date: {end_date}")
print(f"Date range: {date_range}")

In [ ]:
df = spark.sql(f"""
SELECT _token_associate_id AS user_id,
    click_object_id AS item_id,
    SUM(click) AS rating
FROM
    onedata_us_east_1_shared_dit.nas_raw_lyric_search_dit.ml_search_with_click
WHERE
    click_object_id IS NOT NULL AND action = "actions"
GROUP BY
    _token_associate_id,
    click_object_id
""")

In [ ]:
df.toPandas().to_csv("data/input/search_click.csv", index=False)

In [ ]:
df = spark.sql(f"""
SELECT
    click_object_id AS item_id,
    click_details_caption AS title
FROM
    onedata_us_east_1_shared_dit.nas_raw_lyric_search_dit.ml_search_with_click
WHERE
    click_object_id IS NOT NULL AND action = "actions"
GROUP BY
    click_object_id,
    click_details_caption
""")

In [ ]:
df.toPandas().to_csv("data/input/item_desc.csv", index=False)

In [ ]:
df = spark.sql(f"""
SELECT
    _token_associate_id AS user_id,
    click_object_id AS item_id,
    TO_UNIX_TIMESTAMP(time_stamp, 'yyyy-MM-dd\'T\'HH:mm:ss.SSS\'Z\'') AS unix_timestamp,
    SUM(click) AS rating
FROM
    onedata_us_east_1_shared_dit.nas_raw_lyric_search_dit.ml_search_with_click
WHERE
    click_object_id IS NOT NULL AND action = "actions"
GROUP BY
    _token_associate_id,
    click_object_id,
    TO_UNIX_TIMESTAMP(time_stamp, 'yyyy-MM-dd\'T\'HH:mm:ss.SSS\'Z\'')
""")

In [ ]:
df.toPandas().to_csv("data/input/search_click_ts", index=False)

In [ ]:
df = spark.sql(f"""
SELECT
    click_object_id AS item_id,
    click_details_caption AS title,
    concat_ws('|', collect_set(client_id)) AS categories
FROM
    onedata_us_east_1_shared_dit.nas_raw_lyric_search_dit.ml_search_with_click
WHERE
    click_object_id IS NOT NULL and action = "actions"
GROUP BY
    click_object_id,
    click_details_caption
 """)

In [ ]:
df.toPandas().to_csv("data/input/item_desc_clientid", index=False)

In [ ]:
df = spark.sql(f"""
SELECT
    view._token_associate_id AS user_id,
    view._id AS item_id,
    view.click AS rating
FROM
    onedata_us_east_1_shared_dit.nas_raw_lyric_search_dit.ml_search_with_click AS view
JOIN
    (
        SELECT
            traceId,
            MAX(resPos) AS max_resPos
        FROM
            onedata_us_east_1_shared_dit.nas_raw_lyric_search_dit.ml_search_with_click
        WHERE
            click_object_id IS NOT NULL
        GROUP BY
            traceId
    ) AS click
ON
    view.traceId = click.traceId
    AND view.resPos <= click.max_resPos
WHERE view.action = "actions"
 """)

In [ ]:
df.toPandas().to_csv("data/input/view_click.csv", index=False)

In [ ]:
df = spark.sql(f"""
SELECT DISTINCT
    _token_associate_id AS user_id,
    LAST_VALUE(user_agent) OVER (
        PARTITION BY _token_associate_id
        ORDER BY time_stamp
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) AS last_user_agent
FROM
    onedata_us_east_1_shared_dit.nas_raw_lyric_search_dit.ml_search_with_click
 """)

In [ ]:
df.toPandas().to_csv("data/input/user_desc.csv", index=False)

### Load Output Datasets

In [ ]:
df = spark.sql(f"""
SELECT
    *
FROM
    onedata_us_east_1_shared_dit.nas_raw_lyric_search_dit.cold_start_recommended_actions
 """)

In [ ]:
df.toPandas().to_csv("data/output/cold_start.csv", index=False)